In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd


In [4]:
# Seed for reproducibility
np.random.seed(42)

# Simple transaction features: amount, country, merchant
N = 1000
df = pd.DataFrame({
    "amount": np.random.uniform(1, 1000, N),
    "country": np.random.choice([0, 1], N),    # 0=AU, 1=NZ
    "merchant": np.random.choice([0, 1], N),   # 0=ACME_CO, 1=OTHER
    "valid": np.random.randint(0, 2, N)        # target
})

# Features and labels
X = df[['amount', 'country', 'merchant']].values.astype(np.float32)
y = df['valid'].values.astype(np.float32)

# Convert to PyTorch tensors
X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y).unsqueeze(1)

In [5]:
import torch.nn as nn

class SimpleValidator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 8),  # 3 input features → 8 hidden units
            nn.ReLU(),         # activation
            nn.Linear(8, 1),   # hidden → single output
            nn.Sigmoid()       # probability between 0 and 1
        )
        
    def forward(self, x):
        return self.net(x)

# Create model instance
model = SimpleValidator()


In [6]:
from torch.utils.data import DataLoader, TensorDataset

# Wrap dataset for PyTorch
dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function & optimizer
criterion = nn.BCELoss()          # Binary Cross-Entropy
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(20):
    for xb, yb in loader:
        pred = model(xb)               # forward pass
        loss = criterion(pred, yb)     # compute loss
        optimizer.zero_grad()          # reset gradients
        loss.backward()                # backprop
        optimizer.step()               # update weights
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 0.6905
Epoch 2, Loss: 0.7340
Epoch 3, Loss: 0.7002
Epoch 4, Loss: 0.7496
Epoch 5, Loss: 0.7002
Epoch 6, Loss: 0.7695
Epoch 7, Loss: 0.7178
Epoch 8, Loss: 0.6079
Epoch 9, Loss: 0.7291
Epoch 10, Loss: 0.9599
Epoch 11, Loss: 0.8851
Epoch 12, Loss: 0.4197
Epoch 13, Loss: 0.6334
Epoch 14, Loss: 1.2110
Epoch 15, Loss: 0.6211
Epoch 16, Loss: 0.6863
Epoch 17, Loss: 0.9025
Epoch 18, Loss: 0.7682
Epoch 19, Loss: 0.6829
Epoch 20, Loss: 0.6455


In [15]:
import sys
!{sys.executable} -m pip install onnx onnxruntime --upgrade

In [16]:
import torch

# Dummy input with same shape as model input (1 sample, 3 features)
dummy_input = torch.randn(1, 3)

# Path to save ONNX model
onnx_path = "validator_model.onnx"

# Export the PyTorch model to ONNX
torch.onnx.export(
    model,
    dummy_input,
    "validator_model.onnx",
    input_names=['amount','country','merchant'],
    output_names=['valid_prob'],
    opset_version=11,
    dynamo=False  # <-- disable new export logic
)

print(f"Model exported to {onnx_path}")


/var/folders/nh/0x9vjd350jdfdnrwfs09_wpw0000gn/T/ipykernel_81577/2899833745.py:10: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


Model exported to validator_model.onnx
